In [1]:
import time
import hopsworks
import os
from dotenv import load_dotenv
import joblib

# from datetime import datetime, timedelta
import pandas as pd

import hopsworks
import matplotlib.pyplot as plt

from utils import *
import utils


In [2]:
today = int(time.time())
seconds_in_one_day = 24 * 60 * 60
today

1735835435

In [3]:
load_dotenv()
#os.environ["HOPSWORKS_API_KEY"] = os.getenv("HOPSWORKS_API_KEY")
project = hopsworks.login()
#secrets = secrets_api(project.name)
# This line will fail if you have not registered the CRYPTO_API_KEY as a secret in Hopsworks
#CRYPTO_API_KEY = secrets.get_secret("CRYPTO_API_KEY").value

#CRYPTO_API_KEY = os.getenv("CRYPTO_API_KEY")

2025-01-02 18:30:35,049 INFO: Initializing external client
2025-01-02 18:30:35,050 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-02 18:30:38,277 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1160346


In [4]:
fs = project.get_feature_store() 
starting_timestamp = float(today - 40 * seconds_in_one_day)
starting_timestamp


1732379435.0

In [5]:
today, starting_timestamp

(1735835435, 1732379435.0)

In [6]:
solana_feature_group = fs.get_feature_group(
    name="solana",  
    version=1
)

solana_batch_data = solana_feature_group.filter(solana_feature_group.timestamp >= starting_timestamp).read().sort_values(by='timestamp', inplace=False).reset_index(drop=True)
solana_batch_data.head()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.21s) 


,timestamp,open,high,low,close,volume,volume_buy,volume_sell
0,1732406400,255.26,259.96,241.21,252.93,333221.510079,173161.319468,160060.190611
1,1732492800,252.93,256.65,231.32,233.96,430081.090503,191964.534235,238116.556267
2,1732579200,233.96,239.68,221.82,230.68,445887.775346,201503.607457,244384.167889
3,1732665600,230.68,245.00,227.66,242.26,265621.873707,161330.187523,104291.686184
4,1732752000,242.26,245.46,233.00,237.59,142466.637324,73142.398338,69324.238987


In [7]:
bitcoin_feature_group = fs.get_feature_group(
    name="bitcoin",  
    version=1
)
bitcoin_batch_data = bitcoin_feature_group.filter(bitcoin_feature_group.timestamp >= starting_timestamp).read().sort_values(by='timestamp', inplace=False).reset_index(drop=True)
bitcoin_batch_data = bitcoin_batch_data[["timestamp","open"]]
bitcoin_batch_data.head()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.83s) 


,timestamp,open
0,1732406400,97759.1
1,1732492800,97994.1
2,1732579200,93000.0
3,1732665600,91914.5
4,1732752000,95952.9


In [8]:
fng_feature_group = fs.get_feature_group(
    'f_n_g_index',
    version=1
)
fng_batch_data = fng_feature_group.filter(fng_feature_group.timestamp >= starting_timestamp).read().sort_values(by='timestamp', inplace=False).reset_index(drop=True)
fng_batch_data['value_classification'] = fng_batch_data['value_classification'].map(fng_labels_to_numbers_map)
fng_batch_data.head()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.66s) 


,value,value_classification,timestamp
0,80,4,1.732406e+09
1,82,4,1.732493e+09
2,79,4,1.732579e+09
3,75,3,1.732666e+09
4,77,4,1.732752e+09


In [9]:
from tensorflow.keras.models import load_model

mr = project.get_model_registry()

retrieved_model = mr.get_model(
    name="solana_price_daily_model_10d",
    version=3,
)
saved_model_dir = retrieved_model.download()
print(saved_model_dir)
model = load_model(saved_model_dir+'/best_model.keras')
model


/var/folders/rh/xp6gtwpn19s_8lqdxq8sdl0r0000gn/T/5cd04689-7372-4fa3-840f-4fa63e677685/solana_price_daily_model_10d/3


<Sequential name=sequential, built=True>

In [10]:
print(solana_batch_data['timestamp'].dtype)
print(bitcoin_batch_data['timestamp'].dtype)
print(fng_batch_data['timestamp'].dtype)

print(solana_batch_data['timestamp'].head())
print(bitcoin_batch_data['timestamp'].head())
print(fng_batch_data['timestamp'].head())


int64
int64
float64
0    1732406400
1    1732492800
2    1732579200
3    1732665600
4    1732752000
Name: timestamp, dtype: int64
0    1732406400
1    1732492800
2    1732579200
3    1732665600
4    1732752000
Name: timestamp, dtype: int64
0    1.732406e+09
1    1.732493e+09
2    1.732579e+09
3    1.732666e+09
4    1.732752e+09
Name: timestamp, dtype: float64


In [11]:
all_data = pd.merge(solana_batch_data, bitcoin_batch_data, how='inner', on="timestamp",suffixes=('_solana', '_bitcoin'))  
all_data = pd.merge(all_data, fng_batch_data, how='inner', on="timestamp")

if "timestamp" in all_data.columns:
    all_data = all_data.drop(columns=['timestamp'])

target_column = 'open_solana'
all_data = utils.move_target_at_the_end(all_data, target_column)
all_data_values = all_data.values[-30:]

all_data.head()

,high,low,close,volume,volume_buy,volume_sell,open_bitcoin,value,value_classification,open_solana
0,259.96,241.21,252.93,333221.510079,173161.319468,160060.190611,97759.1,80,4,255.26
1,256.65,231.32,233.96,430081.090503,191964.534235,238116.556267,97994.1,82,4,252.93
2,239.68,221.82,230.68,445887.775346,201503.607457,244384.167889,93000.0,79,4,233.96
3,245.00,227.66,242.26,265621.873707,161330.187523,104291.686184,91914.5,75,3,230.68
4,245.46,233.00,237.59,142466.637324,73142.398338,69324.238987,95952.9,77,4,242.26


In [12]:
scaler_labels = joblib.load('scaler_labels.pkl')
scaler_features = joblib.load('scaler_features.pkl')

all_data_values = scaler_features.transform(all_data_values)
X = all_data_values.reshape(1, 30, 10)


In [13]:
all_data.columns

Index(['high', 'low', 'close', 'volume', 'volume_buy', 'volume_sell',
       'open_bitcoin', 'value', 'value_classification', 'open_solana'],
      dtype='object')

In [14]:
predictions = model.predict(X)[0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step


In [15]:
predictions

array([0.4602867 , 0.5461706 , 0.4309681 , 0.38671726, 0.4226027 ,
       0.55054015, 0.5240623 , 0.7281094 , 0.44401348, 0.5068848 ],
      dtype=float32)

In [16]:
range(len(predictions))

range(0, 10)

In [17]:

timestamps = [today + (i+1) * seconds_in_one_day for i in range(len(predictions))]

In [18]:
monitor_fg = fs.get_or_create_feature_group(
    name='solana_predictions',
    description='Solana_predictions',
    version=1,
    primary_key=['date']
)

In [19]:
result = pd.DataFrame(columns=['timestamp', "open_solana"])
result["timestamp"]=timestamps
result["open_solana"]=predictions
#result= result.sort_values(by=['timestamp'])

In [20]:
result

,timestamp,open_solana
0,1735921835,0.460287
1,1736008235,0.546171
2,1736094635,0.430968
3,1736181035,0.386717
4,1736267435,0.422603
5,1736353835,0.550540
6,1736440235,0.524062
7,1736526635,0.728109
8,1736613035,0.444013
9,1736699435,0.506885


In [21]:
result['date'] = pd.to_datetime(result['timestamp'], unit='s')
result['date'] = result['date'].dt.normalize()
result.drop(columns=["timestamp"], inplace=True)

result

,open_solana,date
0,0.460287,2025-01-03
1,0.546171,2025-01-04
2,0.430968,2025-01-05
3,0.386717,2025-01-06
4,0.422603,2025-01-07
5,0.550540,2025-01-08
6,0.524062,2025-01-09
7,0.728109,2025-01-10
8,0.444013,2025-01-11
9,0.506885,2025-01-12


In [22]:
#this is plane and we may have duplicates
#monitor_fg.insert(result)

In [23]:
#this insert prevents duplicates
monitor_fg.insert(
    result,
    #overwrite=False,  # Do not overwrite all data
    operation="upsert",  # Use upsert to handle duplicates based on the primary key
    #storage=None  # Write to both online and offline storage
)

Uploading Dataframe: 100.00% |██████████| Rows 10/10 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: solana_predictions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1160346/jobs/named/solana_predictions_1_offline_fg_materialization/executions


(Job('solana_predictions_1_offline_fg_materialization', 'SPARK'), None)